<a href="https://colab.research.google.com/github/mohith789p/Nvidia/blob/main/OpenCV_Installation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 📦 1. Install Python Tools and Create a Virtual Environment

To isolate OpenCV and its dependencies, we use a **Python virtual environment**. This avoids conflicts with system Python packages and is a good practice for development.

---

#### 🔧 Steps:
1. **Install `pip3`**, the package manager for Python 3.
2. **Install `virtualenv`**, which lets us create isolated Python environments.
3. **Create the virtual environment** named `env`, using the system’s site packages.
4. **Activate the environment**.
5. **Verify that OpenCV is installed later** by importing it.

> ✅ These steps ensure a clean and modular setup.


In [ ]:
sudo apt-get install python3-pip
pip3 install virtualenv
python3 -m virtualenv -p python3 env --system-site-packages
source env/bin/activate
python -c 'import cv2; print(cv2.__version__)'

### 🧠 2. Create Swap File (Important for Jetson Nano)

Jetson Nano has limited RAM (~4GB), and building OpenCV requires more memory. We add a 4GB swap file to avoid memory errors during compilation.

---

#### 🧱 Steps:
1. Allocate a 4GB swap file.
2. Set permissions so only root can access it.
3. Format it as swap space.
4. Enable it immediately with `swapon`.
5. Make it **persist after reboot** by adding an entry to `/etc/fstab`.

> ⚠️ Without this, the OpenCV build may fail with memory-related errors.

In [ ]:
sudo fallocate -l 4G /var/swapfile
sudo chmod 600 /var/swapfile
sudo mkswap /var/swapfile
sudo swapon /var/swapfile
sudo bash -c 'echo "/var/swapfile swap swap defaults 0 0"  >> /etc/fstab'

### 🔄 3. Reboot and Check Swap Space

After enabling the swap file, reboot your Jetson Nano and verify that the swap is active.

---

#### ✅ Check with `free -h`:
This will show you the available memory and swap. You should now see ~4GB of swap space listed.

In [ ]:
sudo reboot
# After reboot
free -h

### 🧩 4. Install OpenCV Dependencies

Before building OpenCV, we must install a set of essential system packages. These include tools and libraries for:

- **Building software** (`cmake`, `build-essential`, `git`)
- **Image and video I/O** (`libjpeg`, `libpng`, `libtiff`, `libavcodec`, `libswscale`, etc.)
- **GUI support** (`libgtk`, `libcanberra-gtk*`)
- **Python bindings** (`python3-dev`, `numpy`, etc.)
- **Video and camera support** (`libv4l`, `gstreamer`)
- **Math acceleration** (`OpenBLAS`, `ATLAS`, `Eigen`, `LAPACK`)
- **HDF5 and protobuf support** (used in OpenCV's DNN module)

> 🛠️ These dependencies ensure OpenCV can interface with media files, perform hardware acceleration, and use cameras or DNNs.

In [ ]:
# Add CUDA library path (specific for Jetson Nano)
sudo sh -c "echo '/usr/local/cuda/lib64' >> /etc/ld.so.conf.d/nvidia-tegra.conf"
sudo ldconfig

# Core build tools
sudo apt-get install build-essential cmake git unzip pkg-config

# Image I/O
sudo apt-get install libjpeg-dev libpng-dev libtiff-dev

# Video I/O
sudo apt-get install libavcodec-dev libavformat-dev libswscale-dev

# GUI support
sudo apt-get install libgtk2.0-dev libcanberra-gtk*

# Python support
sudo apt-get install python3-dev python3-numpy python3-pip

# Codec and GUI extras
sudo apt-get install libxvidcore-dev libx264-dev libgtk-3-dev

# Parallelism and threading
sudo apt-get install libtbb2 libtbb-dev libdc1394-22-dev

# Video capturing and streaming
sudo apt-get install libv4l-dev v4l-utils
sudo apt-get install libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev

# Audio and codec support
sudo apt-get install libavresample-dev libvorbis-dev libxine2-dev
sudo apt-get install libfaac-dev libmp3lame-dev libtheora-dev
sudo apt-get install libopencore-amrnb-dev libopencore-amrwb-dev

# Math libraries
sudo apt-get install libopenblas-dev libatlas-base-dev libblas-dev
sudo apt-get install liblapack-dev libeigen3-dev gfortran

# Miscellaneous
sudo apt-get install libhdf5-dev protobuf-compiler
sudo apt-get install libprotobuf-dev libgoogle-glog-dev libgflags-dev

### 📥 5. Download OpenCV Source Code and Extra Modules

Now we’ll download the OpenCV source code (v4.5.1) and the `opencv_contrib` modules. These contrib modules include extra features such as:

- DNN models
- xfeatures2d (SIFT, SURF)
- Face and text modules

---

After downloading, we:
1. Unzip the folders.
2. Rename them for convenience.
3. Remove the zip files to clean up.

> 📦 Always match the version of `opencv_contrib` with the OpenCV version.

In [ ]:
cd ~
wget -O opencv.zip https://github.com/opencv/opencv/archive/4.5.1.zip
wget -O opencv_contrib.zip https://github.com/opencv/opencv_contrib/archive/4.5.1.zip
unzip opencv.zip
unzip opencv_contrib.zip

# Rename directories
mv opencv-4.5.1 opencv
mv opencv_contrib-4.5.1 opencv_contrib

# Cleanup zip files
rm opencv.zip
rm opencv_contrib.zip

### 🏗️ 6. Configure OpenCV Build with CMake

This step prepares the OpenCV build configuration.

We use `cmake` with many flags:
- `WITH_CUDA=ON`, `WITH_CUDNN=ON`: Enable GPU acceleration.
- `WITH_TBB=ON`: Multithreading with Intel TBB.
- `WITH_GSTREAMER=ON`: Video streaming support.
- `OPENCV_EXTRA_MODULES_PATH`: Adds contrib modules.
- `BUILD_opencv_python3=TRUE`: Ensures Python 3 bindings are built.
- `WITH_OPENGL=ON`, `WITH_V4L=ON`, etc.: Enable camera, GUI, and acceleration features.

> 🚧 This sets up the compilation pipeline for a Jetson-optimized OpenCV build.

In [ ]:
cd ~/opencv
mkdir build
cd build

cmake -D CMAKE_BUILD_TYPE=RELEASE -D CMAKE_INSTALL_PREFIX=/usr -D OPENCV_EXTRA_MODULES_PATH=~/opencv_contrib/modules -D EIGEN_INCLUDE_PATH=/usr/include/eigen3 -D WITH_OPENCL=OFF -D WITH_CUDA=ON -D CUDA_ARCH_BIN=5.3 -D CUDA_ARCH_PTX="" -D WITH_CUDNN=ON -D WITH_CUBLAS=ON -D ENABLE_FAST_MATH=ON -D CUDA_FAST_MATH=ON -D OPENCV_DNN_CUDA=ON -D ENABLE_NEON=ON -D WITH_QT=OFF -D WITH_OPENMP=ON -D WITH_OPENGL=ON -D BUILD_TIFF=ON -D WITH_FFMPEG=ON -D WITH_GSTREAMER=ON -D WITH_TBB=ON -D BUILD_TBB=ON -D BUILD_TESTS=OFF -D WITH_EIGEN=ON -D WITH_V4L=ON -D WITH_LIBV4L=ON -D OPENCV_ENABLE_NONFREE=ON -D INSTALL_C_EXAMPLES=OFF -D INSTALL_PYTHON_EXAMPLES=OFF -D BUILD_NEW_PYTHON_SUPPORT=ON -D BUILD_opencv_python3=TRUE -D OPENCV_GENERATE_PKGCONFIG=ON -D BUILD_EXAMPLES=OFF ..


### ⚙️ 7. Build and Install OpenCV

Now compile OpenCV. This can take **1.5 to 2 hours** on Jetson Nano.

---

#### Notes:
- Use `make -j4` to compile using 4 threads.
- After build, install OpenCV to system directories.
- Remove conflicting headers (important for compatibility).
- Finally, clean up the build files and update shared libraries.

> 💡 Don't interrupt this step. Use a fan or heat sink to keep Nano cool.

In [ ]:
# Compile
make -j4

# Install
cd ~
sudo rm -r /usr/include/opencv4/opencv2
cd ~/opencv/build
sudo make install
sudo ldconfig
make clean
sudo apt-get update

### ✅ 8. Verify OpenCV Installation

Check if OpenCV was successfully installed by importing it in Python.

---

You should see the version number printed without any errors.

In [ ]:
import cv2
print(cv2.__version__)

### 📊 9. Install `jtop` for Jetson Monitoring

`jtop` is like `htop`, but tailored for Jetson devices. It shows:

- CPU and GPU usage
- Memory, temperature, and power stats
- Real-time monitoring of system health

> 🚀 Essential during long builds like OpenCV or running deep learning models.

In [ ]:
cd ~
sudo -H pip3 install -U jetson-stats
sudo reboot

> After reboot, you can start monitoring with:

In [ ]:
jtop

### 📷 10. Test Camera on Jetson Nano

Verify that your CSI or USB camera is working using NVIDIA's `nvgstcapture-1.0` tool.

---

#### Detect Camera Devices:

In [ ]:
ls /dev/video0       # CSI camera
ls /dev/video*       # List all video devices

# Capture Photo Using CSI Camera
nvgstcapture-1.0 --orientation=2

# Capture Photo Using USB Camera (V4L2)
nvgstcapture-1.0 --camsrc=0 --cap-dev-node=1

## ✅ Recap: What We Achieved

Here's a summary of the complete setup done on Jetson Nano:

1. Installed Python tooling and created a virtual environment
2. Added 4GB swap space to support large builds
3. Installed all required OpenCV dependencies
4. Downloaded OpenCV (v4.5.1) and contrib modules
5. Configured the OpenCV build with CUDA, cuDNN, and hardware optimizations
6. Compiled and installed OpenCV with Python bindings
7. Verified installation using `cv2.__version__`
8. Installed `jtop` for system monitoring
9. Tested CSI/USB camera using NVIDIA tools

> You're now ready to run OpenCV projects with GPU acceleration on Jetson Nano!